### **Detectron2 installation**

In [ ]:
!python -m pip install pyyaml==5.1
!pip install CocoDataset==0.1.2
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities.
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]

### **Import libraries**

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow
import copy

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog



In [ ]:
im = cv2.imread("/content/bus/COCO_train2014_000000049254.jpg")
cv2_imshow(im)

### **Download dataset** - REMOVE!

In [ ]:
# args = sys.argv
# args = ["http://images.cocodataset.org/annotations/annotations_trainval2014.zip",
#         "/content/annotations_trainval2014.zip",
#         "http://images.cocodataset.org/zips/train2014.zip",
#         "/content/train2014.zip"
#         ]

args = ["http://images.cocodataset.org/annotations/annotations_trainval2014.zip",
        "/content/annotations_trainval2014.zip",
        "http://images.cocodataset.org/zips/train2014.zip",
        "/content/train2014.zip"
        ]

annotations_link = args[0]
annotations_unzip_path = args[1]
images_link = args[2]
images_unzip_path = args[3]

!wget "http://images.cocodataset.org/annotations/annotations_trainval2014.zip"
!unzip "/content/annotations_trainval2014.zip"

!wget "http://images.cocodataset.org/zips/train2014.zip"
!unzip "/content/train2014.zip"

!wget "http://images.cocodataset.org/zips/val2014.zip"
!unzip "/content/val2014.zip"

# from coco_dataset import coco_dataset_download as cocod
# class_name='bus'  #class name example 
# # images_count=50       #count of images  
# annotations_path= '/content/annotations/instances_train2014.json' #path of coco dataset annotations 
# #call download function 
# cocod.coco_dataset_download(class_name,images_count,annotations_path)

### Data Augmentation class


*   Sky cropping
*   List item



In [ ]:
class Data_Augmentation:

  def __init__(self):
    self.segmentation_pred = None
    # self.detection_pred = None
    self.data_set = None

  def __predictor_init(self):
    cfg = get_cfg()
    # initialize segementation model
    cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
    self.segmentation_pred = DefaultPredictor(cfg)
    # initialize detection model
    # cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    # cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
    # cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    # self.detection_pred = DefaultPredictor(cfg)
  
  def sky_crop(self, img_dict):
    self.__predictor_init()
    img = cv2.imread(img_dict["file_name"])
    # run inference on image
    panoptic_seg, segments_info = self.segmentation_pred(img)["panoptic_seg"]
    # outputs = self.detection_pred(img)
    # finding label corresponding to sky
    id = 0
    for dict in segments_info:
      if dict['category_id'] == 40:
        id = dict['id']
    if id == 0:
      print("no sky detected")
      self.__update_seg(0, img_dict['annotations'])
      return img_dict
    # creating mask on sky
    panoptic_seg = panoptic_seg.cpu()
    mask = np.where(panoptic_seg==id, 1, 0)
    # finding max sky pixel
    y_ind, x_ind = np.where(mask==1)
    max_y = y_ind[-1]
    # cropping image
    cropped_img = img[max_y:, :, :]
    # Save the image
    image_name = "/content/sky_cropped/new_image" + str(img_dict['image_id']) + ".jpg"
    path = 'sky_cropped'
    os.makedirs(path, exist_ok=True)
    cv2.imwrite(image_name, cropped_img)
    img_dict['file_name'] = image_name
    img_dict['height'] = cropped_img.shape[0]
    img_dict['width'] = cropped_img.shape[1]
    # fix bboxes
    img_dict['annotations'] = self.__update_boxes(max_y, img_dict['annotations'])
    self.__update_seg(max_y, img_dict['annotations'])
    return img_dict
    
  def __update_boxes(self, max_y, boxes):
    new_dicts = []
    # going through all dicts and for each one update bbox
    for i in range(len(boxes)):
      x_min, y_min, width, height = boxes[i]['bbox']  
      if max_y <= y_min:
        boxes[i]['bbox'][1] = int(y_min - max_y)
        new_dicts.append(boxes[i])
        continue
      if max_y < y_min+height:
        boxes[i]['bbox'][3] = int(y_min+height - max_y)
        boxes[i]['bbox'][1] = 0
        new_dicts.append(boxes[i])
    return new_dicts

  def __update_seg(self, max_y, boxes):
  # new_dicts = []
  # going through all dicts and for each one update bbox
    for i in range(len(boxes)):
      boxes[i]['segmentation'] = []
      # arr = boxes[i]['segmentation']
      # for j in range(0, len(arr[0])-2, 2): 
      #   # print(j)
      #   curr_y = arr[0][j] 
      #   if max_y <= curr_y:
      #     arr[0][j] = curr_y - max_y
      #     # new_dicts.append(boxes[i])
      #     continue
      #   if curr_y < max_y:
      #     arr[0][j] = 0
          # new_dicts.append(boxes[i])
    # return new_dicts
      
        

## Registering coco dataset 
assuming this dataset is in COCO format



In [ ]:
from detectron2.structures import BoxMode
from detectron2.data.datasets import register_coco_instances
args = [
        "/content/annotations/instances_train2014.json",
        "/content/train2014",
        "My_dataset2",
        "/content/sky_cropped/new_image.jpg",
        "/content/annotations/sky_crop_annotations.json"
        ]

path_to_annotations = args[0]
path_to_images = args[1]
dataset_name = args[2]
path_to_new_images = args[3]
path_to_new_annotations = args[4]
register_coco_instances(dataset_name, {}, path_to_annotations, path_to_images)

# Loading annotations

In [ ]:
from detectron2.data.datasets import load_coco_json
# Load the image and annotation data for the "my_dataset2_train" dataset
dataset_dicts = load_coco_json(path_to_annotations, path_to_images, dataset_name)
# Get the first image and annotation in the dataset
metadata = MetadataCatalog.get(dataset_name)

# Single photo example

In [ ]:
import matplotlib.pyplot as plt
d = None
for i in range(len(dataset_dicts)):
  if dataset_dicts[i]["file_name"] == "/content/train2014/COCO_train2014_000000000049.jpg":
    d = copy.deepcopy(dataset_dicts[i])
    print("Found it!")
    break
aug = Data_Augmentation()
curr_dict = dataset_dicts[i]
new_dict = aug.sky_crop(d)
img = cv2.imread(new_dict['file_name'])
annotations = new_dict['annotations']
visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1)
out = visualizer.draw_dataset_dict(new_dict)
cv2_imshow(out.get_image()[:, :, ::-1])


multiple images

In [ ]:
# for every dict from the list perform sky_crop then vizualize
aug = Data_Augmentation()
for i in range(len(dataset_dicts)):
  curr_dict = dataset_dicts[i]
  new_dict = aug.sky_crop(curr_dict)
  img = cv2.imread(curr_dict['file_name'])
  visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1)
  out = visualizer.draw_dataset_dict(curr_dict)
  cv2_imshow(out.get_image()[:, :, ::-1])

# Saving json

In [ ]:
with open(path_to_new_annotations, "w") as f:
    # Write the list of dictionaries to the file as JSON
    json.dump(dataset_dicts, f)